In [ ]:
import pywt
import numpy as np
from matplotlib.pyplot import *
import pickle
from joblib import Parallel, delayed

EQ=np.loadtxt('EQ.csv')*9.81*39.30
import os as os
results=[]
def wavel(erqt,lvl,wve,num,real,av):
    coeffs = pywt.wavedec(erqt, wve, level=lvl)
    with open("coeffs/coeffs{}".format(num), "wb") as fp:
        pickle.dump(coeffs, fp)
    
    cf=coeffs[0]
    rd_coeff=np.zeros((len(cf),real))
    cnt=num*real
    count_file=[]
    for i in range(real):
        rd=np.random.uniform(-av,av,len(cf))
        coeffs[0]=rd*cf
        rd_coeff[:,i]=rd*cf
        eq_real=pywt.waverec(coeffs,wve)
#         np.savetxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/load_files/Load.{}.{}.txt".format(num,i)), eq_real)
        np.savetxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/load_files/Load.{}.txt".format(cnt)), eq_real)
        cnt=cnt+1
        count_file=np.append(count_file,cnt)
    np.savetxt("coeffs/rd_coeff_{}.txt".format(num), rd_coeff)
#     np.savetxt("coeffs/count_file{}.txt".format(num), count_file)
#     return rd_coeff



def Ndof_stif(num_real,Ndof,rd_seed,dta_set,cov):
    
    Es=29000
    fcE=1.0
#     MyCol=1.75e4
#     MyBeam= 1.2e4 
    MyCol=2e4
    MyBeam= 1.5e4 
    
    
    Mn=np.array([Es,MyCol, MyBeam])
    std=cov*Mn*0.01
    
    STF=np.empty((num_real,0))
    np.random.seed(rd_seed)
    for i in range(Ndof):
        np.random.seed(rd_seed+i)
        
#         stf=np.transpose(np.array([np.random.uniform(Mn[0]-std[0],Mn[0]+std[0],num_real),np.random.uniform(Mn[1]-std[1],Mn[1]+std[1],num_real),np.random.uniform(Mn[2]-std[2],Mn[2]+std[2],num_real)]))
        stf=np.transpose(np.array([np.random.uniform(Mn[0]-fcE*std[0],Mn[0]+fcE*std[0],num_real),np.random.uniform(Mn[1]-std[1],Mn[1]+std[1],num_real),np.random.uniform(Mn[2]-std[2],Mn[2]+std[2],num_real)]))
        
        #         print(stf[0,0])
        
        STF=np.append(STF,stf,axis=1)
#     4STF=STF.T   
    if dta_set=='training':
        np.savetxt("Stiffnes{}Dof.txt".format(Ndof), STF)
        np.savetxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/Stiffnes{}Dof.txt".format(Ndof)), STF)
    elif dta_set=='IDA':
        np.savetxt("Stiffnes{}Dof_IDA.txt".format(Ndof), STF)
        np.savetxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/Stiffnes{}Dof_IDA.txt".format(Ndof)), STF)
    else:
        np.savetxt("Stiffnes{}Dof_1eq.txt".format(Ndof), STF)
        np.savetxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/Stiffnes{}Dof_1eq.txt".format(Ndof)), STF)
    return STF

        

In [ ]:
level=4
real=15
Ndof=1
np.savetxt('Ndof.txt',np.array([Ndof]))
np.savetxt(os.path.expanduser('~/OpenSees/Opensees_ml_runs/Ndof.txt'),np.array([Ndof]))

rd_seed=42
cov=20
av=3


results = Parallel(n_jobs=120)(delayed(wavel)(EQ[:,i],level,'sym3',i,real,av) for i in range(len(EQ.T)))  
# with open("coeffs/rd_coeffs", "wb") as fp:
#         pickle.dump(results, fp)

# rd_c=[]
# for i in range(len(EQ.T)):
#     rd_c_temp=np.loadtxt('coeffs/rd_coeff_{}.txt'.format(i))
#     rd_c=np.append(rd_c,rd_c_temp)

Stf=Ndof_stif(real*len(EQ.T),Ndof,rd_seed,'training',cov) 
lst=[]
for i in range(len(EQ.T)):
    rd_c_temp=np.loadtxt('coeffs/rd_coeff_{}.txt'.format(i))
    lst.append(rd_c_temp)
lst_ff=np.array(lst)
lst_dd=lst_ff.transpose(0,2,1)
shp=lst_dd.shape
rd_c=lst_dd.reshape(shp[0]*shp[1],shp[2])
np.savetxt('wv_coeff.txt',rd_c)

In [ ]:
# # rl=1900
# fct=np.linspace(0.1,2,real)
# rd_coeff_IDA=np.zeros((len(rd_c.T),len(EQ.T)*len(fct),))
# def wavel_IDA(lvl,wve,real):
#     for j in range(len(EQ.T)):
#         sig=fct[real]*EQ[:,j]
#         coeffs = pywt.wavedec(sig, wve, level=lvl)
#         np.savetxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/load_files/Load_IDA.{}.txt".format(real*42+j)), sig)
#         rd_coeff_IDA[:,real*42+j]=coeffs[0]
# # results = Parallel(n_jobs=42)(delayed(wavel_IDA)(level,'sym3',rl) for rl in range(len(fct)))  
# for rl in range(len(fct)):
#     r=wavel_IDA(level,'sym3',rl)
# np.savetxt('wv_coeff_IDA.txt',rd_coeff_IDA)

In [ ]:
Stf=Ndof_stif(real*len(EQ.T),Ndof,rd_seed,'IDA',cov) 

In [ ]:
# nm=1
# ida=np.loadtxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/load_files/Load_IDA.{}.txt".format(rl*42+nm)))
# plot(ida)
# plot(EQ[:,nm])
# xlim([0,1500])
# # plot(ida)




In [ ]:
# rd_c=np.empty_like(np.loadtxt('coeffs/rd_coeff_{}.txt'.format(1)))
# rd_c=rd_c.T

# for i in range(len(EQ.T)):
#     rd_c_temp=np.loadtxt('coeffs/rd_coeff_{}.txt'.format(i))
#     rd_c=np.append(rd_c,rd_c_temp.T,axis=0)

In [ ]:
#check
# q=rd_c[7,:] for 7 realizations
# p=lst[1][:,0]
# p-q

In [ ]:
# num=3
# def fftrans(dt,sig):
#     Fs = 1/dt 
#     NFFT=np.power(2,np.ceil(np.log2(len(sig))))
#     df=Fs/NFFT
#     #frequencies=2*np.pi*np.arange(0,Fs-df,df)
#     frequencies=2*np.pi*np.arange(0,Fs,df)
#     #fourierTransform = np.fft.fft(sig,n=int(NFFT))
#     fourierTransform = np.fft.fft(sig,n=int(NFFT))/len(sig)           # Normalize amplitude

# #     fourierTransform = fourierTransform[0:int(NFFT/2)+1]

# #     fourierTransform[1:len(fourierTransform)-1]=2*fourierTransform[1:len(fourierTransform)-1]

#     freq=frequencies[0:len(fourierTransform)]
#     amp=np.abs(fourierTransform)
#     return freq,amp,fourierTransform
# for i in range(8):
#     figure(i+1)
# #     plot(EQ[:,num])
#     d=np.loadtxt(os.path.expanduser("~/OpenSees/Opensees_ml_runs/load_files/Load.{}.{}.txt".format(1,i)))
# #     plot(d,'--')


#     freq,amp1,fourierTransform=np.array(fftrans(0.01,EQ[:,num]))
#     figure(i)
#     plot(amp1[:int(len(amp1)/2)])
#     xlim([0,1000])
#     freq,amp2,fourierTransform=np.array(fftrans(0.01,d))
#     # figure(2)
#     plot(amp2[:int(len(amp2)/2)])
#     xlim([0,1000])

In [ ]:
# import numpy as np
# import os as os
# import pywt
# from matplotlib.pyplot import *
# EQ=np.loadtxt('EQ.csv')
# for i in range(42):
#     figure(i)
#     erqt=EQ[:,i]
#     coeffs = pywt.wavedec(erqt, 'sym3', level=4)
#     plot(erqt)
    
